In [34]:
!pip install EIA_python
import numpy as np 
import pandas as pd
import json
import eia
from patsy import dmatrices
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [4]:
apiKey = "5f54b3e66477e22ec068066b1de8026d"
api = eia.API(apiKey)

In [7]:
url = "http://api.eia.gov/geoset/?geoset_id=sssssss&regions=region1,region2,region3,...&api_key={0}[&start=|&num=][&end=][&out=xml|json]".format(api)

In [35]:
series_id_list = ["INTL.57-1-DZA-TBPD.M", "INTL.57-1-AGO-TBPD.M", "INTL.57-1-COG-TBPD.M", "INTL.57-1-COD-TBPD.M", 
                  "INTL.57-1-ECU-TBPD.M","INTL.57-1-GNQ-TBPD.M","INTL.57-1-GAB-TBPD.M","INTL.57-1-IRN-TBPD.M",
                  "INTL.57-1-IRQ-TBPD.M", "INTL.57-1-KWT-TBPD.M", "INTL.57-1-LBY-TBPD.M", "INTL.57-1-NGA-TBPD.M",
                  "INTL.57-1-QAT-TBPD.M", "INTL.57-1-RUS-TBPD.M", "INTL.57-1-SAU-TBPD.M","INTL.57-1-ARE-TBPD.M",
                  "INTL.57-1-VEN-TBPD.M", "INTL.57-1-USA-TBPD.M"]

In [36]:
df_list = [pd.DataFrame(api.data_by_series(series)) for series in series_id_list]

In [37]:
oil_data = pd.concat(df_list, axis=1)

In [38]:
oil_data = oil_data.replace("--", np.nan)

In [39]:
oil_data.dropna().shape

(330, 18)

In [40]:
oil_data.shape

(570, 18)

In [41]:
oil_data_reduced = oil_data.dropna()

In [42]:
oil_data_reduced

,"Crude oil including lease condensate production, Algeria, Monthly (thousand barrels per day)","Crude oil including lease condensate production, Angola, Monthly (thousand barrels per day)","Crude oil including lease condensate production, Congo-Brazzaville, Monthly (thousand barrels per day)","Crude oil including lease condensate production, Congo-Kinshasa, Monthly (thousand barrels per day)","Crude oil including lease condensate production, Ecuador, Monthly (thousand barrels per day)","Crude oil including lease condensate production, Equatorial Guinea, Monthly (thousand barrels per day)","Crude oil including lease condensate production, Gabon, Monthly (thousand barrels per day)","Crude oil including lease condensate production, Iran, Monthly (thousand barrels per day)","Crude oil including lease condensate production, Iraq, Monthly (thousand barrels per day)","Crude oil including lease condensate production, Kuwait, Monthly (thousand barrels per day)","Crude oil including lease condensate production, Libya, Monthly (thousand barrels per day)","Crude oil including lease condensate production, Nigeria, Monthly (thousand barrels per day)","Crude oil including lease condensate production, Qatar, Monthly (thousand barrels per day)","Crude oil including lease condensate production, Russia, Monthly (thousand barrels per day)","Crude oil including lease condensate production, Saudi Arabia, Monthly (thousand barrels per day)","Crude oil including lease condensate production, United Arab Emirates, Monthly (thousand barrels per day)","Crude oil including lease condensate production, Venezuela, Monthly (thousand barrels per day)","Crude oil including lease condensate production, United States, Monthly (thousand barrels per day)"
1993 01,1195.0,484.857,181.0,25.0,328.0,4.5,319.0,3539.997,500.0,1657.102,1462.806,2032.0,449.688,7194.074,8500.0,2211.030,2484.343,6961.007
1993 02,1195.0,505.273,181.0,25.0,328.0,4.5,319.0,3636.983,500.0,1845.072,1408.445,2013.0,429.701,7180.238,8440.0,2220.664,2463.726,6942.592
1993 03,1175.0,510.376,181.0,25.0,328.0,4.5,319.0,3588.490,500.0,1632.369,1334.316,1984.0,399.722,7088.008,8300.0,2186.945,2412.183,6974.128
1993 04,1175.0,544.061,181.0,25.0,328.0,4.5,319.0,3394.518,500.0,1627.422,1334.316,1936.0,399.722,7069.563,8000.0,2186.945,2412.183,6881.308
1993 05,1175.0,544.061,181.0,25.0,343.0,4.5,319.0,3539.997,500.0,1694.201,1334.316,1936.0,419.708,6912.766,8000.0,2148.408,2412.183,6846.920
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020 02,1240.0,1392.000,293.0,14.0,537.0,153.0,200.0,2842.000,4688.0,2889.000,200.000,1959.0,1530.000,10897.000,9802.0,3508.000,840.000,12745.586
2020 03,1260.0,1372.000,282.0,14.0,541.0,163.0,200.0,2778.000,4537.0,3081.000,150.000,1939.0,1530.000,10894.000,9852.0,3814.000,670.000,12737.065
2020 04,1260.0,1342.000,293.0,14.0,210.0,168.0,190.0,2695.000,4537.0,3330.000,135.000,1889.0,1530.000,10935.000,11653.0,4128.000,620.000,12009.967
2020 05,1060.0,1272.000,283.0,14.0,335.0,133.0,180.0,2676.000,4257.0,2378.000,130.000,1809.0,1530.000,8993.000,8603.0,2825.000,545.000,10016.161


In [43]:
oil_data_reduced.columns = ["Algeria", "Angola", "Congo-Brazzaville", "Congo-Kinshasa", "Ecuador", "Equatorial Guinea", "Gabon", "Iran",
                "Iraq", "Kuwait", "Libya", "Nigeria", "Qatar", "Russia", "Saudi Arabia", "United Arab Emirates", "Venezuela", "USA"]
corrs = oil_data_reduced.corr()

In [44]:

corrs[(corrs>=.5) | (corrs<=-.5) & (corrs!=1)]

,Algeria,Angola,Congo-Brazzaville,Congo-Kinshasa,Ecuador,Equatorial Guinea,Gabon,Iran,Iraq,Kuwait,Libya,Nigeria,Qatar,Russia,Saudi Arabia,United Arab Emirates,Venezuela,USA
Algeria,1.000000,0.633219,NaN,NaN,0.605699,0.899932,NaN,NaN,NaN,NaN,NaN,0.695220,NaN,0.527458,NaN,NaN,NaN,NaN
Angola,0.633219,1.000000,NaN,-0.690092,0.812523,0.735502,-0.770910,NaN,0.634853,0.794322,NaN,NaN,0.914164,0.891764,0.675151,0.736283,NaN,NaN
Congo-Brazzaville,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Congo-Kinshasa,NaN,-0.690092,NaN,1.000000,-0.692394,-0.602615,0.891680,NaN,-0.764917,-0.764011,NaN,NaN,-0.807572,-0.865942,-0.672947,-0.756335,0.725738,NaN
Ecuador,0.605699,0.812523,NaN,-0.692394,1.000000,0.741539,-0.759360,NaN,0.632089,0.831444,NaN,NaN,0.803295,0.862663,0.726879,0.723702,NaN,NaN
Equatorial Guinea,0.899932,0.735502,NaN,-0.602615,0.741539,1.000000,-0.677975,NaN,NaN,0.522016,NaN,0.644163,0.613225,0.685314,NaN,NaN,NaN,NaN
Gabon,NaN,-0.770910,NaN,0.891680,-0.759360,-0.677975,1.000000,NaN,-0.748842,-0.782512,NaN,NaN,-0.851564,-0.930824,-0.697139,-0.752918,0.654324,NaN
Iran,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Iraq,NaN,0.634853,NaN,-0.764917,0.632089,NaN,-0.748842,NaN,1.000000,0.772425,-0.537197,NaN,0.814844,0.760436,0.706892,0.835843,-0.610206,0.697909
Kuwait,NaN,0.794322,NaN,-0.764011,0.831444,0.522016,-0.782512,NaN,0.772425,1.000000,NaN,NaN,0.869363,0.901167,0.915799,0.950613,-0.601576,0.594993


In [ ]:
oil_data_reduced.to_stata("oil_data_bootcamp.dta")